# Building a deep research agent with the OLLAMA client


In [ ]:
!uv add ollama

In [6]:
from ollama import Client
from openai import OpenAI

openai_client = OpenAI(
    base_url='http://192.168.1.177:11434/v1',
    api_key='ollama'
)

client = Client(
  host='http://192.168.1.177:11434',
)

In [7]:
import requests
import json

url = "http://jarvita-agx:11434/api/tags"

response = requests.get(url)
for tag in response.json()['models']:
    # print(f"{tag}")
    print(f"-------{tag['name']}-------")
    for i,prop in tag.items():
        print(f"{prop}")


-------deepseek-r1:32b-------
deepseek-r1:32b
deepseek-r1:32b
2025-12-28T13:21:44.346123968-05:00
19851337809
edba8017331d15236e57480eb45406c0d721db77a4cdcf234df500fc2ad3960c
{'parent_model': '', 'format': 'gguf', 'family': 'qwen2', 'families': ['qwen2'], 'parameter_size': '32.8B', 'quantization_level': 'Q4_K_M'}
-------deepseek-r1:70b-------
deepseek-r1:70b
deepseek-r1:70b
2025-12-28T12:35:08.487301622-05:00
42520397873
d37b54d01a76512a5adbaa9da4ee1c00833bf2dfefb0d4cc54351ba529e0f4da
{'parent_model': '', 'format': 'gguf', 'family': 'llama', 'families': ['llama'], 'parameter_size': '70.6B', 'quantization_level': 'Q4_K_M'}
-------nemotron-3-nano:30b-------
nemotron-3-nano:30b
nemotron-3-nano:30b
2025-12-28T00:42:54.956573741-05:00
24271934866
b725f11174073334edd0c2ff396b8d4e66d7dab22a5a63717ccad5a08a270cf1
{'parent_model': '', 'format': 'gguf', 'family': 'nemotron_h_moe', 'families': ['nemotron_h_moe'], 'parameter_size': '31.6B', 'quantization_level': 'Q4_K_M'}
-------qwen3-vl:8b-------

In [8]:
model_info = client.show("gemma3n:e2b")
for info in model_info:
    print(f"{info}")


('modified_at', datetime.datetime(2025, 9, 8, 22, 38, 56, 911790, tzinfo=TzInfo(-14400)))
('template', '{{- range $i, $_ := .Messages }}\n{{- $last := eq (len (slice $.Messages $i)) 1 }}\n{{- if or (eq .Role "user") (eq .Role "system") }}<start_of_turn>user\n{{ .Content }}<end_of_turn>\n{{ if $last }}<start_of_turn>model\n{{ end }}\n{{- else if eq .Role "assistant" }}<start_of_turn>model\n{{ .Content }}{{ if not $last }}<end_of_turn>\n{{ end }}\n{{- end }}\n{{- end }}')
('modelfile', '# Modelfile generated by "ollama show"\n# To build a new Modelfile based on this, replace FROM with:\n# FROM gemma3n:e2b\n\nFROM /ssd/ollama/models/blobs/sha256-3839a254cf2d00b208c6e2524c129e4438f9d106bba4c3fbc12b631f519d1de1\nTEMPLATE """{{- range $i, $_ := .Messages }}\n{{- $last := eq (len (slice $.Messages $i)) 1 }}\n{{- if or (eq .Role "user") (eq .Role "system") }}<start_of_turn>user\n{{ .Content }}<end_of_turn>\n{{ if $last }}<start_of_turn>model\n{{ end }}\n{{- else if eq .Role "assistant" }}<star

In [9]:
models = client.list()
for model in models['models']:
    print(f"-------{type(model)}-------")
    model_info = client.show(model['model'])
    for key, value in model:
        print(f"{key}: {value}")
    print(model_info['capabilities'])
    # for key, value in model:
    #     print(value)


-------<class 'ollama._types.ListResponse.Model'>-------
model: deepseek-r1:32b
modified_at: 2025-12-28 13:21:44.346123-05:00
digest: edba8017331d15236e57480eb45406c0d721db77a4cdcf234df500fc2ad3960c
size: 19851337809
details: parent_model='' format='gguf' family='qwen2' families=['qwen2'] parameter_size='32.8B' quantization_level='Q4_K_M'
['completion', 'thinking']
-------<class 'ollama._types.ListResponse.Model'>-------
model: deepseek-r1:70b
modified_at: 2025-12-28 12:35:08.487301-05:00
digest: d37b54d01a76512a5adbaa9da4ee1c00833bf2dfefb0d4cc54351ba529e0f4da
size: 42520397873
details: parent_model='' format='gguf' family='llama' families=['llama'] parameter_size='70.6B' quantization_level='Q4_K_M'
['completion', 'thinking']
-------<class 'ollama._types.ListResponse.Model'>-------
model: nemotron-3-nano:30b
modified_at: 2025-12-28 00:42:54.956573-05:00
digest: b725f11174073334edd0c2ff396b8d4e66d7dab22a5a63717ccad5a08a270cf1
size: 24271934866
details: parent_model='' format='gguf' fami

In [ ]:
# model="granite4:latest"
# model = "gpt-oss:20b"
model = "qwen3"

## Create an agent that likes to keep the conversation going

In [16]:
response = client.chat(model="gemma3n:e2b", messages=[
  {'role': 'user', 'content': 'Why is the sky blue? use low reasoning and make it short'},
])

In [17]:
print(response.message.content)

The sun's light is made of all colors. When it enters the Earth's air, blue light gets scattered more than other colors. That's why we see a blue sky! 

**(Basically, blue light bounces around more than other colors.)** 



In [ ]:
from ollama import chat

def get_temperature(city: str) -> str:
  """Get the current temperature for a city
  
  Args:
    city: The name of the city

  Returns:
    The current temperature for the city
  """
  temperatures = {
    "New York": "22°C",
    "London": "15°C",
    "Tokyo": "18°C",
  }
  return temperatures.get(city, "Unknown")

messages = [{"role": "user", "content": "What's the temperature in New York?"}]

# pass functions directly as tools in the tools list or as a JSON schema
response = client.chat(model=model, messages=messages, tools=[get_temperature], think=True)

messages.append(response.message)
if response.message.tool_calls:
  # only recommended for models which only return a single tool call
  call = response.message.tool_calls[0]
  result = get_temperature(**call.function.arguments)
  # add the tool result to the messages
  messages.append({"role": "tool", "tool_name": call.function.name, "content": str(result)})

  final_response = client.chat(model=model, messages=messages, tools=[get_temperature], think=True)
  print(final_response.message.content)

In [19]:
import pydantic
from pydantic import BaseModel
# Structured Output Class
class Country(BaseModel):
  name: str
  capital: str
  languages: list[str]

response = client.chat(
  model=model,
  messages=[{'role': 'user', 'content': 'Tell me about Canada.'}],
  format=Country.model_json_schema(),
)

country = Country.model_validate_json(response.message.content)
print(country)

ValidationError: 1 validation error for ChatRequest
model
  Input should be a valid string [type=string_type, input_value=Model(model='gemma3n:e2b'...ization_level='Q4_K_M')), input_type=ListResponse.Model]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type

In [ ]:
system_prompt = """
You are a helpful assistant that can answer questions and help with tasks.
Please answer in bullet points and limit it to four bullet points with 2 sentances each.
"""

response = client.chat.completions.create(
    model="gpt-oss:20b",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": ""}
    ]
)
 
print(response.choices[0].message.content)


In [ ]:
result = await agent.run("How can reputation resolutsion help my company?")
print(result.output)

## Create an agent with memory with previous message memory

In [ ]:
result = await agent.run("What can reputation resolutsion do for me and give me 2 blog posts from them to read up on?")
print(result.output)

In [ ]:
result2 = await agent.run("What company are we talking about?", message_history=result.all_messages())
print(result2.output)

### Longer Chat History Memory - WIP

In [ ]:
memory = []

In [ ]:
result = await agent.run("What is Data Science Dojo?")
print(result.output)
memory.append(result.all_messages())

In [ ]:
print(type(result))
for i in result.all_messages():
    print(i)

In [ ]:
result = await agent.run("Tell me about the company apple")
print(result.output)
memory.append(result.all_messages())

In [ ]:
result.all_messages()

In [ ]:
result2 = await agent.run("What company are we talking about?", message_history=result.all_messages())
print(result2.output)

In [ ]:
memory

In [ ]:
result

In [ ]:
result = await agent.run("What company are we talking about?", message_history=memory)
print(result.output)

## Connect to Wevaite

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

# WEAVIATE_URL = os.getenv("WEAVIATE_URL")
# WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")


# print("Weaviate URL:", WEAVIATE_URL)
# print("Weaviate API Key:", WEAVIATE_KEY[:10])
print("OpenAI API Key:", OPENAI_API_KEY[:10])

In [ ]:
import weaviate
from weaviate.classes.init import Auth

client = weaviate.connect_to_local(
    # cluster_url=WEAVIATE_URL,
    # auth_credentials=Auth.api_key(WEAVIATE_KEY),
    headers = {
        "X-OpenAI-Api-Key": OPENAI_API_KEY,
        "X-Cohere-Api-Key": COHERE_API_KEY
    },
)

print("Client ready:", client.is_ready())

In [ ]:
# Check our collection from before
contracts = client.collections.use("ReputationresolutionsWebsite")
contracts_config = contracts.config.get()

print(contracts_config)

## Create a query enhancement agent

This agent will take the users question and optimize it for the search to the vector database (Weaviate)

In [ ]:
from pydantic_ai import Agent

# Simple query optimizer and search agent
query_enhancement_agent = Agent(
    model="openai:gpt-4o-mini",
    instructions="You optimize queries for contract search, then search and return results.",
)

async def search_(user_query):
    # Optimize and search in one step
    result = await query_enhancement_agent.run(f"Optimize this query for search again a reputation management website: {user_query}")
    optimized_query = result.output

    # Search the database
    response = contracts.query.hybrid(query=optimized_query, limit=3)

    # Show results
    for i, contract in enumerate(response.objects):
        print(f"Page Title {i+1}: {contract.properties['page_title']}")

    return response.objects



In [ ]:
# Test it
results = await search_("How can reputation resolutsion help my company?")

In [ ]:
from weaviate.classes.query import Filter
from datetime import datetime, timezone

# Get collection properties
collection_config = contracts.config.get()
properties = {prop.name: str(prop.data_type) for prop in collection_config.properties}

# Smart filtering agent that creates filters automatically
smart_filter_agent = Agent(
    model="openai:gpt-4o-mini",
    instructions=f"""
    You analyze queries and automatically create Weaviate filters.
    Collection properties: {properties}
    
    Based on the user query, generate Python code that creates Filter objects.
    Only return the filter code, nothing else.
    
    Examples:
    - For "Jane Doe contracts": Filter.by_property("author").equal("Jane Doe")
    - For "recent employment contracts": Filter.by_property("contract_type").equal("employment contract") & 
Filter.by_property("date").greater_than(datetime(2023, 6, 1, tzinfo=timezone.utc))
    - For "short contracts": Filter.by_property("contract_length").less_than(2)
    
    If no filters needed, return: None
    """,
)

async def auto_filtered_search(user_query):
    # Get filter code from agent
    filter_result = await smart_filter_agent.run(f"Create filters for: {user_query}")
    filter_code = filter_result.output.strip()

    print(f"Query: {user_query}")
    print(f"Generated filter: {filter_code}")

    # Execute the filter code
    query_filters = None
    if filter_code != "None":
        try:
            query_filters = eval(filter_code)
        except:
            print("Filter creation failed, searching without filters")

    # Search with auto-generated filters
    response = contracts.query.near_text(
        query=user_query,
        filters=query_filters,
        limit=5
    )

    print(f"\nFound {len(response.objects)} pages:")
    for i, contract in enumerate(response.objects):
        print(f"Document {i+1}: {contract.properties['page_title']}")

    return response.objects

In [ ]:
results = await auto_filtered_search("give me information about the use cases deloitte is talking about")

# Put it all together

We will do a query expansion and then run through our dynamic filter agentm

In [ ]:
from weaviate.classes.query import Filter
from datetime import datetime, timezone

def auto_filtered_search(user_query: str)-> str:
    # Get collection properties
    collection_config = contracts.config.get()
    properties = {prop.name: str(prop.data_type) for prop in collection_config.properties}

    # Smart filtering agent that creates filters automatically
    smart_filter_agent = Agent(
        model="openai:gpt-4o-mini",
        instructions=f"""
        You analyze queries and automatically create Weaviate filters.
        Collection properties: {properties}
        
        Based on the user query, generate Python code that creates Filter objects.
        Only return the filter code, nothing else.
        
        Examples:
        - For "Jane Doe contracts": Filter.by_property("author").equal("Jane Doe")
        - For "recent employment contracts": Filter.by_property("contract_type").equal("employment contract") & 
    Filter.by_property("date").greater_than(datetime(2023, 6, 1, tzinfo=timezone.utc))
        - For "short contracts": Filter.by_property("contract_length").less_than(2)
        
        If no filters needed, return: None
        """,
    )


    # Get filter code from agent
    filter_result = smart_filter_agent.run(f"Create filters for: {user_query}")
    filter_code = filter_result.output.strip()

    print(f"Query: {user_query}")
    print(f"Generated filter: {filter_code}")

    # Execute the filter code
    query_filters = None
    if filter_code != "None":
        try:
            query_filters = eval(filter_code)
        except:
            print("Filter creation failed, searching without filters")

    # Search with auto-generated filters
    response = contracts.query.near_text(
        query=user_query,
        filters=query_filters,
        limit=5
    )

    print(f"\nFound {len(response.objects)} contracts:")
    for i, contract in enumerate(response.objects):
        print(f"Document {i+1}: {contract.properties['document_title']} page {contract.properties['page']}")

    return response.objects

In [ ]:
# planner_agent.py
from pydantic_ai import Agent
# from basic_tools import smart_filter_tool, query_enhancement_tool

system = """
You are a planning agent.
You can use two tools:
1. smart_filter_tool(query) — filters or constrains a query.
2. query_enhancement_tool(query) — expands or improves a query.

Decide when to call each tool (or both, in sequence) to best improve a user query.
Always output the final improved query as plain text.
"""
research_agent = Agent(
    model="openai:gpt-4o",
    system_prompt=system,
    tools=[auto_filtered_search],
    )



In [ ]:
research_agent_resul = await research_agent.run(f"tell me about use cases on page 7 from deloitte")

In [ ]:
client.close()